## DLT Pipeline

#### Importing Libraries

In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
%sql
SELECT * FROM databricks_catalog.silver.ext_silver_products

#### Streaming Table

In [0]:
@dlt.table

def DimProducts_stg():
    df = spark.readStream.table('databricks_catalog.silver.ext_silver_products')
    df = df.withColumn('DimProductKey', monotonically_increasing_id()+lit(0))
    return df

#### Streaming View

In [0]:
@dlt.view

def DimProducts_view():
    df = spark.readStream.table('Live.DimProducts_stg')
    return df

#### DimProducts

In [0]:
dlt.create_streaming_table('DimProducts')

In [0]:
dlt.apply_changes(
  target = "DimProducts",
  source = "DimProducts_view",
  keys = ['product_id'],
  sequence_by= col("product_id"),
  stored_as_scd_type=2
)